In [15]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision import transforms

## 读入数据

数据来自 BCI Competition III Dataset II, P300 打字机。

分类任务：给定一段脑电信号，判断闪光的是哪个字母。

In [2]:
from scipy.io import loadmat

dataset = loadmat("dataset/Subject_B_Train.mat")

In [7]:
signals = dataset['Signal']
flashing = dataset['Flashing']
types = dataset['StimulusType'] # 0 when intensified row / column does not include target character.
target_chars = dataset['TargetChar']

signals.shape

(85, 7794, 64)

In [27]:
# Transpose the shape into (C, T)
features = torch.transpose(torch.from_numpy(signals), 1, 2)
features.shape

torch.Size([85, 64, 7794])

In [51]:
target_chars = list(dataset['TargetChar'][0])

In [58]:
import pandas as pd

mapping_table = pd.read_excel("mapping.xlsx", header=None)
mapping = dict()

for key, value in mapping_table.values:
    mapping[str(key)] = value

mapping

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25,
 '1': 26,
 '2': 27,
 '3': 28,
 '4': 29,
 '5': 30,
 '6': 31,
 '7': 32,
 '8': 33,
 '9': 34,
 '_': 35}

In [86]:
labels = torch.Tensor(list(map(lambda ch: mapping[ch], target_chars))).long()
labels

tensor([21,  6, 17,  4,  0,  0,  7, 33, 19, 21, 17,  7,  1, 24, 13, 35, 20,  6,
         2, 14, 11, 14, 29,  4, 20,  4, 17,  3, 14, 14,  7,  2,  8,  5, 14, 12,
         3, 13, 20, 31, 11, 16,  2, 15, 10,  4,  8, 17,  4, 10, 14, 24, 17, 16,
         8,  3,  9, 23, 15,  1, 10, 14,  9,  3, 22, 25,  4, 20,  4, 22, 22,  5,
        14,  4,  1,  7, 23, 19, 16, 19, 19, 25, 20, 12, 14])

In [82]:
one_hot_labels = F.one_hot(labels.long(), 36)
one_hot_labels[0]

AttributeError: 'Tensor' object has no attribute 'astype'

In [87]:
class EEGDataset(Dataset):
    def __init__(self, features: torch.Tensor, labels: torch.Tensor):
        super().__init__()
        self.features = features.unsqueeze(1) # Transfer into 3D
        self.labels = labels
    
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx: int):
        return self.features[idx], self.labels[idx]

p300_dataset = EEGDataset(features, labels)

In [88]:
trans = transforms.ToTensor()
loader = DataLoader(p300_dataset, batch_size=1, shuffle=True) # 构造数据集

loader.__iter__().__next__()

[tensor([[[[  9.1361,   7.8425,   3.9616,  ...,  -9.2331,  -6.9046, -11.8203],
           [  5.2344,   0.3977,  -2.6570,  ...,  -1.1296,  -1.3842,  -6.7299],
           [  6.5554,   2.7274,   1.4514,  ...,  -5.4390,  -5.1838,  -9.2669],
           ...,
           [  3.7801,   7.0978,  11.1810,  ..., -13.8287, -18.4223, -17.6567],
           [  9.5049,  11.8247,  14.6601,  ..., -11.3737, -13.9513, -16.2711],
           [  6.7405,   9.2781,  10.0394,  ...,  -6.4550, -10.7689,  -9.7539]]]]),
 tensor([2])]

In [22]:
class MaxNormConv2d(nn.Conv2d):
    def __init__(self, *args, max_norm=1, **kwargs):
        super().__init__(*args, **kwargs)
        self.max_norm = max_norm
    
    def forward(self, x):
        self.weight.data = torch.renorm(
            self.weight.data, p=2, dim=0, maxnorm=self.max_norm
        )
        return super().forward(x) 


In [29]:
class EEGNet(nn.Module):
    def __init__(self, num_channels):
        super(EEGNet, self).__init__()
        self.T = 120 # Half of sample frequency
    
        self.C = num_channels
        self.F1 = 16 # The number of channels after first conv2d
        self.D = 2 # depth multiplier, used in depthwise conv
        self.p = 0.5 # dropout rate
        self.F2 = self.F1 * self.D
        self.N = 36 # number of classes

        # Here input shape is (1, C, T)
        b1 = nn.Sequential(
            nn.Conv2d(1, self.F1, (1, 64), padding="same"), # 1D Convolution to time
            nn.BatchNorm2d(self.F1, affine=True, eps=1e-3),
            MaxNormConv2d(self.F1, self.F1 * self.D,
            (self.C, 1), max_norm=1, groups=self.F1), # Depthwise Conv
            nn.BatchNorm2d(self.F1 * self.D, affine=True, eps=1e-3),
            nn.ELU(),
            nn.AvgPool2d((1, 4)),
            nn.Dropout(self.p)
        )

        b2 = nn.Sequential(
            nn.Conv2d(self.F1 * self.D, self.F1 * self.D, (1, 16),
                groups = self.F1 * self.D, padding="same"), # Separable Conv 1 (depthwise conv)
            nn.Conv2d(self.F1 * self.D, self.F2, kernel_size=1), # Separable Conv 1 (pointwise conv)
            nn.BatchNorm2d(self.F2),
            nn.ELU(),
            nn.AvgPool2d((1, 8)),
            nn.Dropout(self.p),
            nn.Flatten()
        )

        self.net = nn.Sequential(b1, b2, nn.LazyLinear(self.N))
    
    def forward(self, x):
        return self.net(x)

In [30]:
net = EEGNet(num_channels=64)
y1 = net(torch.from_numpy(signals[0]).T.reshape(1, 1, 64, -1))

D:\program\Python\anaconda\envs\d2l\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
D:\program\Python\anaconda\envs\d2l\lib\site-packages\torch\nn\modules\conv.py:453: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:883.)
  return F.conv2d(input, weight, bias, self.stride,


In [48]:
torch.from_numpy(signals).unsqueeze(0).shape # Add a dimension

loader.__iter__().__next__()[0].shape

torch.Size([1, 1, 64, 7794])

## 开始训练

In [90]:
lr = 0.1
epochs = 50

In [94]:
cuda = torch.device("cuda")
net.to(cuda)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

In [96]:
for epoch in range(epochs):
    for x, y in loader:
        x = x.to(cuda)
        y = y.to(cuda)
        l = loss(net(x), y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    # Here we can test the accuracy
    print(f"{epoch + 1} / {epochs}")

1 / 50
2 / 50
3 / 50
4 / 50
5 / 50
6 / 50
7 / 50
8 / 50
9 / 50
10 / 50
11 / 50
12 / 50
13 / 50
14 / 50
15 / 50
16 / 50
17 / 50
18 / 50
19 / 50
20 / 50
21 / 50
22 / 50
23 / 50
24 / 50
25 / 50
26 / 50
27 / 50
28 / 50
29 / 50
30 / 50
31 / 50
32 / 50
33 / 50
34 / 50
35 / 50
36 / 50
37 / 50
38 / 50
39 / 50
40 / 50
41 / 50
42 / 50
43 / 50
44 / 50
45 / 50
46 / 50
47 / 50
48 / 50
49 / 50
50 / 50
